In [1]:
import pandas as pd

run = pd.read_pickle("evaluations/testdev_sr_clipbase_owllarge/results_snapshot.pkl")
run_with_skips = run
run = run[~run["skipped"]]

In [2]:
import re 

for i, row in run.iterrows():
    with open(f"../data/encoded_questions/{row['question_id']}.lp") as f:
        question_encoding = f.read()
    
    run.loc[i, "has_select"] = bool(re.search("select\(", question_encoding))
    run.loc[i, "has_filter"] = bool(re.search("filter\(", question_encoding))
    run.loc[i, "has_filter_any"] = bool(re.search("filter_any\(", question_encoding))
    run.loc[i, "has_relate"] = bool(re.search("relate\(", question_encoding)) 
    run.loc[i, "has_relate_any"] = bool(re.search("relate_any\(", question_encoding))
    run.loc[i, "has_relate_attr"] = bool(re.search("relate_attr\(", question_encoding))
    run.loc[i, "has_compare"] = bool(re.search("compare\(", question_encoding))
    run.loc[i, "has_query_attr"] = bool(re.search("query\(\w+, \w+, name\)", question_encoding))
    run.loc[i, "has_query_name"] = bool(re.search("query\(\w+, \w+, (?!name)\w+\)", question_encoding))
    run.loc[i, "has_verify_attr"] = bool(re.search("verify_attr\(", question_encoding))
    run.loc[i, "has_verify_rel"] = bool(re.search("verify_rel\(", question_encoding))
    run.loc[i, "has_choose_attr"] = bool(re.search("choose_attr\(", question_encoding))
    run.loc[i, "has_choose_rel"] = bool(re.search("choose_rel\(", question_encoding))
    
    run.loc[i, "has_exist"] = bool(re.search("exist\(", question_encoding))
    run.loc[i, "has_two_same"] = bool(re.search("two_same\(", question_encoding))
    run.loc[i, "has_two_different"] = bool(re.search("two_different\(", question_encoding))
    run.loc[i, "has_all_same"] = bool(re.search("all_same\(", question_encoding))
    run.loc[i, "has_all_different"] = bool(re.search("all_different\(", question_encoding))
    run.loc[i, "has_common"] = bool(re.search("common\(", question_encoding))
    run.loc[i, "has_and"] = bool(re.search("and\(", question_encoding))
    run.loc[i, "has_or"] = bool(re.search("or\(", question_encoding))

/tmp/ipykernel_1678995/3510796415.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run.loc[i, "has_select"] = bool(re.search("select\(", question_encoding))
/tmp/ipykernel_1678995/3510796415.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run.loc[i, "has_filter"] = bool(re.search("filter\(", question_encoding))
/tmp/ipykernel_1678995/3510796415.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [3]:
op_types = [t[4:] for t in list(run) if t.startswith("has_")]
percentages = []

# question type
percentages.append({
    "name": "all",
    "correct": run[run["correct"]].shape[0] / run.shape[0]
})
binary = run[(run["answer"] == "no") | (run["answer"] == "yes")]
percentages.append({
    "name": "binary",
    "correct": binary[binary["correct"]].shape[0] / binary.shape[0]
})
open = run[(run["answer"] != "no") & (run["answer"] != "yes")]
percentages.append({
    "name": "open",
    "correct": open[open["correct"]].shape[0] / open.shape[0]
})

# operations
for op in op_types:
    percentages.append({
        "name": op,
        "correct": run[run[f"has_{op}"] & run["correct"]].shape[0] / max(run[run[f"has_{op}"]].shape[0], 1)
    })
    
results = pd.DataFrame(percentages)
results["correct"] = results["correct"] * 100
print(results.style.format(precision=2).hide(axis="index").to_latex())


\begin{tabular}{lr}
name & correct \\
all & 40.55 \\
binary & 56.87 \\
open & 31.31 \\
select & 40.55 \\
filter & 42.65 \\
filter_any & 40.92 \\
relate & 37.82 \\
relate_any & 8.77 \\
relate_attr & 24.00 \\
compare & 39.13 \\
query_attr & 23.99 \\
query_name & 33.64 \\
verify_attr & 56.48 \\
verify_rel & 62.94 \\
choose_attr & 61.71 \\
choose_rel & 66.35 \\
exist & 56.85 \\
two_same & 50.00 \\
two_different & 41.30 \\
all_same & 76.19 \\
all_different & 60.00 \\
common & 5.21 \\
and & 53.17 \\
or & 52.80 \\
\end{tabular}



In [19]:
run[run["has_compare"]][["answer", "model_response"]]

,answer,model_response
118,muffin,UNSAT
382,boy,[woman]


In [25]:
run[run["correct"]].shape[0] / run.shape[0]

0.40289121202221495

In [5]:
run["runtime_sec"].mean()

0.9938848002138715

In [10]:
run["model_response"] = run["model_response"].str[0]

/tmp/ipykernel_466951/2138938923.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run["model_response"] = run["model_response"].str[0]


In [3]:
long_run = run[run["runtime_sec"] > 4]

In [7]:
# operations
op_types = [t[4:] for t in list(run) if t.startswith("has_")]
for op in op_types:
    print(f"{op}: " + str(long_run[long_run[f"has_{op}"]].shape[0] / long_run.shape[0] * 100) + "%")

select: 100.0%
filter: 16.917293233082706%
filter_any: 2.2556390977443606%
relate: 58.64661654135338%
relate_any: 76.31578947368422%
relate_attr: 0.0%
compare: 0.0%
query_attr: 76.31578947368422%
query_name: 0.0%
verify_attr: 0.0%
verify_rel: 0.0%
choose_attr: 0.0%
choose_rel: 0.0%
exist: 0.0%
two_same: 0.0%
two_different: 0.0%
all_same: 0.0%
all_different: 0.0%
common: 23.684210526315788%
and: 0.0%
or: 0.0%


In [18]:
long_run[long_run["has_relate_any"] | long_run["has_common"]].shape[0]

266

In [19]:
run_base = pd.read_pickle("evaluations/testdev_base_clipbase_owllarge/results_snapshot.pkl")
run_base = run_base[~run_base["skipped"]]

In [13]:
run_base = run_base[:1778]

In [22]:
run[~run["correct"] & run_base["correct"]]["semantic_str"].tolist()

/tmp/ipykernel_1291481/68689323.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  run[~run["correct"] & run_base["correct"]]["semantic_str"].tolist()


['select: table (2)->relate: utensil,on top of,s (1) [0]->verify color: black [1]->verify cleanliness: clean  [1]->and:  [2, 3]',
 'select: cake (4)->verify rel: platter,on,o (-) [0]',
 'select: flag (17)->relate: vehicle,in front of,s (16) [0]->query: name [1]',
 'select: flag (17)->relate: vehicle,in front of,s (16) [0]->query: name [1]',
 'select: net (10)->verify rel: man,in front of,o (8) [0]',
 'select: door (5)->select: kitten (1) ->common:  [0, 1]',
 'select: wall (10)->filter material: wood [0]->relate: refrigerator,in front of,s (0) [1]->exist: ? [2]',
 'select: grass (5)->relate: person,near,s (1) [0]->verify rel: bench,sitting on,o (8) [1]',
 'select: cabinet (9)->filter material: wood [0]->choose rel: books,to the left of|to the right of,s (2) [1]',
 'select: countertop (17)->relate: sink,on,s (3) [0]->relate: _,under,s (13) [1]->query: name [2]',
 'select: bench (6)->verify rel: woman,in front of,o (9) [0]',
 'select: building (12)->verify rel: trees,in front of,o (13) [0

In [18]:
run["correct"]

1       False
2       False
3        True
4       False
5        True
        ...  
1795     True
1796    False
1797    False
1798     True
1799    False
Name: correct, Length: 1778, dtype: bool